In [53]:
import tensorflow as tf

BATCH_SIZE = 100
NUM_CLASSES = 10
LEARNING_RATE = 0.001
IMG_DIM = 28*28
NUM_EPOCHS = 40

# Prepare data given a numpy array of 
def prepDataset(ds):
  d = tf.data.Dataset.from_tensor_slices(ds)
  d = d.map(lambda x, y: (tf.cast(tf.reshape(x, [IMG_DIM]), tf.float32), tf.cast(y, tf.int32)))
  d = d.batch(BATCH_SIZE)
  iter = d.make_initializable_iterator()
  return iter, iter.get_next()


# Evaluate model with test data
# sess: session runtime
def evalTestData(sess):
  sess.run(test_iter.initializer)
  tc, steps = 0, 0
  while True:
    try:
      test_x, test_y = sess.run((test_nx, test_ny))
      c, l = sess.run((correct, loss), feed_dict={x: test_x, y: test_y})
      losses.append(l)
      tc += c
      steps += 1
    except tf.errors.OutOfRangeError:
      avg_loss = sum(losses) / len(losses)
      print("TEST: batch size: %d, steps %d, avg loss %f, precision: %f" % (BATCH_SIZE, steps, avg_loss, tc/(steps*BATCH_SIZE)*100))
      break

# Load data using keras.
(train_iter, (train_nx, train_ny)), (test_iter, (test_nx, test_ny)) = map(prepDataset, tf.keras.datasets.mnist.load_data())

# DEFINE MODEL, building graph
x, y = tf.placeholder(tf.float32, shape=(BATCH_SIZE, IMG_DIM)), tf.placeholder(tf.int32, shape=(BATCH_SIZE))
# first hidden layer
h1_layer = tf.layers.Dense(units=128, activation=tf.nn.relu)
h1 = h1_layer(x)
# second hidden layer
h2_layer = tf.layers.Dense(units=32, activation=tf.nn.relu)
h2 = h2_layer(h1)
# softmax/logit layer
logit_layer = tf.layers.Dense(units=NUM_CLASSES)
y_ = logit_layer(h2)

# eval for testing phase
correct = tf.reduce_sum(tf.cast(tf.nn.in_top_k(y_, y, 1), tf.int32))

# cross entropy loss
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

# Gradient descent optimizer
optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
train_op = optimizer.minimize(loss)

# Runtime of tensorflow: Session
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  # Epoch loop
  for epoch in range(NUM_EPOCHS):
    print('-'*20 + ' Epoch #%d ' % epoch + '-'*20)
    sess.run(train_iter.initializer)
    losses, steps = [], 0
    # Process batches
    while True:
      try:
        train_x, train_y = sess.run((train_nx, train_ny))
        _, l = sess.run((train_op, loss), feed_dict={x: train_x, y: train_y})
        losses.append(l)
        steps += 1
      except tf.errors.OutOfRangeError:
        avg_loss = sum(losses) / len(losses)
        print("TRAIN: batch size: %d, steps: %d, avg loss: %f." % (BATCH_SIZE, steps, avg_loss))
        evalTestData(sess)
        break


-------------------- Epoch #0 --------------------
TRAIN: batch size: 100, steps: 600, avg loss: 2.014851.
TEST: batch size: 100, steps 100, avg loss 1.896175, precision: 68.620000
-------------------- Epoch #1 --------------------
TRAIN: batch size: 100, steps: 600, avg loss: 1.025673.
TEST: batch size: 100, steps 100, avg loss 1.020274, precision: 74.990000
-------------------- Epoch #2 --------------------
TRAIN: batch size: 100, steps: 600, avg loss: 0.883755.
TEST: batch size: 100, steps 100, avg loss 0.884142, precision: 77.120000
-------------------- Epoch #3 --------------------
TRAIN: batch size: 100, steps: 600, avg loss: 0.797388.
TEST: batch size: 100, steps 100, avg loss 0.800991, precision: 78.890000
-------------------- Epoch #4 --------------------
TRAIN: batch size: 100, steps: 600, avg loss: 0.733704.
TEST: batch size: 100, steps 100, avg loss 0.740849, precision: 79.400000
-------------------- Epoch #5 --------------------
TRAIN: batch size: 100, steps: 600, avg loss